In [2]:
import pandas as pd
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import pickle

In [3]:
# Importazione dizionario
with open("Export\Dizionario Metriche\Minimized Version\dicMetric_Fairness_Completed.pkl", "rb") as file:
    dic = pickle.load(file)

In [4]:
df = pd.DataFrame(dic)

In [5]:
# Importazione X_test per prendere nomi variabili e nomi valori
with open('Export\Dizionario Metriche\Minimized Version\X_test_MIN_perPlot.pkl', 'rb') as file:
    X_test = pickle.load(file)

In [6]:
sensitive_variables=['gender', 'age_group', 'disability', 'nationality', 'education_cv_encoded']

In [7]:
sensitive_features = X_test[sensitive_variables]

In [8]:
valori_univoci = []

In [9]:
for col in sensitive_variables:
    for unique_value in X_test[col].unique():
        key = f"{col}_{unique_value}"  # Creare la chiave con il prefisso
        valori_univoci.append(key)  # Salvare il nome della chiave nella lista

In [10]:
def plot_radar_chart(df, groups=None):
    if groups is None:
        groups = [col for col in df.columns if col != "Metric"]
    fig = go.Figure()
    
    for group in groups:
        valid_data = df[group].notna()  # Filtra solo i valori disponibili
        fig.add_trace(go.Scatterpolar(
            r=df.loc[valid_data, group],
            theta=df.loc[valid_data, "Metric"],
            fill='toself',
            name=group
        ))
    
    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True)),
        showlegend=True,
        title="Metriche Aggregate/Dettagliate"
    )
    return fig

In [11]:
# Radar chart per le metriche aggregate
aggregated_fig = plot_radar_chart(df, groups=sensitive_variables)

# Radar chart per le metriche dettagliate
detailed_fig = plot_radar_chart(df, groups=valori_univoci)

In [12]:
# Creazione dell'app JupyterDash
app = JupyterDash(__name__)

# Layout dell'app
app.layout = html.Div([
    dcc.RadioItems(
        id='view-selector',
        options=[
            {'label': 'Aggregated', 'value': 'aggregated'},
            {'label': 'Detailed', 'value': 'detailed'}
        ],
        value='aggregated',
        inline=True
    ),
    dcc.Graph(id='radar-chart', config={'displayModeBar': False})
])

c:\Users\marco\Documents\UniPI\Tesi\PJ\S0_MidBias\Synthetic_cv-job_generator\.conda\lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [13]:
# Callback per aggiornare il grafico
@app.callback(
    Output('radar-chart', 'figure'),
    [Input('view-selector', 'value')]
)
def update_chart(view):
    if view == 'aggregated':
        return aggregated_fig
    elif view == 'detailed':
        return detailed_fig

# Avvio del server Dash nel notebook
app.run_server(mode="inline")

In [14]:
import plotly.io as pio
pio.write_html(aggregated_fig, file='plot_interattivo_agg.html', auto_open=True)
pio.write_html(detailed_fig, file='plot_interattivo_det.html', auto_open=True)